In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.interpolate import griddata

# Preprocessing Data (Filter out unwanted columns and combining symptons into a list in merged_symptons)
CSV_Data = '../data/2021VAERSDATA.csv'
CSV_Symptoms = '../data/2021VAERSSYMPTOMS.csv'
CSV_Vax = "../data/2021VAERSVAX.csv" 

df_d = pd.read_csv(CSV_Data, names = ['VAERS_ID','RECVDATE','STATE','AGE_YRS','CAGE_YR','CAGE_MO','SEX','RPT_DATE','SYMPTOM_TEXT','DIED','DATEDIED','L_THREAT','ER_VISIT','HOSPITAL','HOSPDAYS','X_STAY','DISABLE','RECOVD','VAX_DATE','ONSET_DATE','NUMDAYS','LAB_DATA','V_ADMINBY','V_FUNDBY','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','SPLTTYPE','FORM_VERS','TODAYS_DATE','BIRTH_DEFECT','OFC_VISIT','ER_ED_VISIT','ALLERGIES'],error_bad_lines=False, index_col=False, dtype='unicode')
df = df_d.drop(['CAGE_YR','STATE', 'CAGE_MO', 'DIED','RPT_DATE','SYMPTOM_TEXT','DATEDIED','L_THREAT','ER_VISIT','HOSPITAL','HOSPDAYS','X_STAY','DISABLE','RECOVD','VAX_DATE','ONSET_DATE','NUMDAYS','LAB_DATA','V_ADMINBY','V_FUNDBY','OTHER_MEDS','SPLTTYPE','FORM_VERS','TODAYS_DATE','BIRTH_DEFECT','OFC_VISIT','ER_ED_VISIT'], axis=1)
df_s = pd.read_csv(CSV_Symptoms, names = ['VAERS_ID','SYMPTOM1','SYMPTOMVERSION1','SYMPTOM2','SYMPTOMVERSION2','SYMPTOM3','SYMPTOMVERSION3','SYMPTOM4','SYMPTOMVERSION4','SYMPTOM5','SYMPTOMVERSION5'],error_bad_lines=False, index_col=False, dtype='unicode')
df_s = df_s.drop(['SYMPTOMVERSION1','SYMPTOMVERSION2','SYMPTOMVERSION3','SYMPTOMVERSION4','SYMPTOMVERSION5'], axis = 1)
df_s['MERGED_SYMPTONS'] = df_s[df_s.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df_s = df_s.drop(['VAERS_ID','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5'], axis = 1)
df = pd.concat([df,df_s], axis=1, join="inner")
df_v = pd.read_csv (CSV_Vax, names = ['VAERS_ID','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_ROUTE','VAX_SITE','VAX_NAME'],error_bad_lines=False, index_col=False, dtype='unicode')
df_v = df_v.drop(['VAERS_ID','VAX_TYPE','VAX_LOT','VAX_ROUTE','VAX_SITE','VAX_NAME'], axis = 1)
df = pd.concat([df,df_v], axis=1, join="inner")

df = df.drop (df.index[0])
# Get symptons from age, sex, vac manu, vax dose series
df_as = df.drop(['RECVDATE','CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'ALLERGIES'], axis = 1)
df_as

,VAERS_ID,AGE_YRS,SEX,MERGED_SYMPTONS,VAX_MANU,VAX_DOSE_SERIES
1,0916600,33.0,F,"Dysphagia,Epiglottitis",MODERNA,1
2,0916601,73.0,F,"Anxiety,Dyspnoea",MODERNA,1
3,0916602,23.0,F,"Chest discomfort,Dysphagia,Pain in extremity,V...",PFIZER\BIONTECH,1
4,0916603,58.0,F,"Dizziness,Fatigue,Mobility decreased",MODERNA,UNK
5,0916604,47.0,F,"Injection site erythema,Injection site pruritu...",MODERNA,1
...,...,...,...,...,...,...
146618,1272609,71.0,M,"Blood glucose increased,Fatigue,Headache,Type ...",PFIZER\BIONTECH,2
146619,1272610,41.0,F,"Injection site reaction,Thrombosis",PFIZER\BIONTECH,1
146620,1272636,24.0,F,Death,PFIZER\BIONTECH,2
146621,1272661,87.0,F,"Chest pain,Cough,Rash,Rash erythematous",PFIZER\BIONTECH,1


In [2]:
df_as.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146622 entries, 1 to 146622
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   VAERS_ID         146622 non-null  object
 1   AGE_YRS          131782 non-null  object
 2   SEX              146622 non-null  object
 3   MERGED_SYMPTONS  146622 non-null  object
 4   VAX_MANU         146622 non-null  object
 5   VAX_DOSE_SERIES  145639 non-null  object
dtypes: object(6)
memory usage: 7.8+ MB


In [3]:
# Get symptoms occurrences 

SYMPTOM_Dict ={}
num = 0
for x in df_as['MERGED_SYMPTONS']:
    string = ""
    for char in x:
        if(char == ","):
            if string in SYMPTOM_Dict :
                SYMPTOM_Dict[string] += 1
            else :
                SYMPTOM_Dict[string] = 1
            
            string = ""
        else:
            string += char
    if string in SYMPTOM_Dict :
        SYMPTOM_Dict[string] += 1
    else :
        SYMPTOM_Dict[string] = 1
    
sorted_dict = {}
sorted_keys = sorted(SYMPTOM_Dict, key=SYMPTOM_Dict.get, reverse = True)

# Get symptoms with occurences over 1000
for x in sorted_keys:
    sorted_dict[x] = SYMPTOM_Dict[x]

stop_index = 0
for i, x in enumerate(sorted_dict):
    if sorted_dict[x] > 1000:
        pass
    else:
        stop_index = i
        break

symptoms_to_use = list(sorted_dict.keys())[:stop_index]


symptoms_to_use.remove("SARS-CoV-2 test negative")
symptoms_to_use.remove("SARS-CoV-2 test positive")
symptoms_to_use.remove("SARS-CoV-2 test")
symptoms_to_use.remove("Poor quality product administered")
symptoms_to_use.remove("Death")
symptoms_to_use.remove("COVID-19")
symptoms_to_use.remove("Body temperature")
symptoms_to_use.remove("Product storage error")
symptoms_to_use.remove("Blood test")
symptoms_to_use.remove("Abdominal pain upper")


    
    
print("Symptoms with occurences over 1000: ",symptoms_to_use)

Symptoms with occurences over 1000:  ['Headache', 'Chills', 'Pyrexia', 'Fatigue', 'Pain', 'Nausea', 'Dizziness', 'Pain in extremity', 'Injection site pain', 'Injection site erythema', 'Myalgia', 'Arthralgia', 'Injection site swelling', 'Pruritus', 'Injection site pruritus', 'Rash', 'Dyspnoea', 'Erythema', 'Injection site warmth', 'Vomiting', 'Asthenia', 'Diarrhoea', 'Urticaria', 'Paraesthesia', 'Lymphadenopathy', 'Malaise', 'Hyperhidrosis', 'Hypoaesthesia', 'Injection site rash', 'Feeling abnormal', 'Cough', 'Peripheral swelling', 'Chest discomfort', 'Heart rate increased', 'Skin warm', 'Tremor', 'Body temperature increased', 'Flushing', 'Feeling hot', 'Palpitations', 'Swelling', 'Rash erythematous', 'Chest pain', 'Back pain', 'Decreased appetite', 'Neck pain', 'Paraesthesia oral', 'Oropharyngeal pain', 'Injection site induration', 'Vaccination site pain', 'Mobility decreased', 'Rash pruritic', 'Tachycardia', 'Feeling cold', 'Blood pressure increased', 'Electrocardiogram', 'Influenza l

In [4]:
# add symptoms column to dataframe
symtoms = symptoms_to_use

for symtom in symtoms:
    l = []
    num = 0
    for x in df_as['MERGED_SYMPTONS']:
        string = ""
        check_contain = False
        for char in x:
            if(char == ","):
                if string == symtom:
                    check_contain = True
                string = ""
            else:
                string += char
        if string == symtom:
            check_contain = True
        num += 1
        if (check_contain):
            l.append(1)
        else:
            l.append(0)
    df_as.insert(len(df_as.columns),symtom,l,True)

df_asn = df_as.drop(["MERGED_SYMPTONS"],axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder_SEX = LabelEncoder()
encoder_VAX_MANU = LabelEncoder()


# Remove rows where VAX_DOSE_SERIES is not 1 or 2
df_asn.drop(df_asn[(df_asn['VAX_DOSE_SERIES']!="1") & (df_asn['VAX_DOSE_SERIES']!="2")].index, inplace = True)
# Remove rows where VZX_MANU is not Moderna, Pfizer\Biotech, or Janssen
df_asn.drop(df_asn[(df_asn['VAX_MANU']!="MODERNA") & (df_asn['VAX_MANU']!="PFIZER\BIONTECH") &(df_asn['VAX_MANU']!="JANSSEN")].index, inplace = True)
# ID_encoded = le.fit_transform(np.ravel(df_as["VAERS_ID"]))
df_asn["VAERS_ID"] = df_asn["VAERS_ID"].astype(str).astype(int)
# Remove rows where Age is missing
df_asn.drop(df_asn[df_asn['AGE_YRS'].isna()].index, inplace = True)
df_asn["AGE_YRS"] = df_asn["AGE_YRS"].astype(str).astype(float)
Sex_encoded = encoder_SEX.fit_transform(np.ravel(df_asn["SEX"]))
# 0 = Female, 1 = Male
df_asn["SEX"] = Sex_encoded
# 0 = JANSSEN, 1 = MODERNA, 2 = PFIZER\BIONTECH
VAX_MANU_encoded = encoder_VAX_MANU.fit_transform(np.ravel(df_asn["VAX_MANU"]))
df_asn["VAX_MANU"] = VAX_MANU_encoded
df_asn["VAX_DOSE_SERIES"] = df_asn["VAX_DOSE_SERIES"].astype(str).astype(int)

df_asn

,VAERS_ID,AGE_YRS,SEX,VAX_MANU,VAX_DOSE_SERIES,Headache,Chills,Pyrexia,Fatigue,Pain,...,Condition aggravated,Lethargy,Hypoaesthesia oral,Loss of consciousness,Hypertension,Abdominal pain upper,Injection site reaction,Musculoskeletal stiffness,Abdominal pain,Muscular weakness
1,916600,33.0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,916601,73.0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916602,23.0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,916604,47.0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,916606,44.0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146618,1272609,71.0,1,2,2,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
146619,1272610,41.0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
146620,1272636,24.0,0,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146621,1272661,87.0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_asn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106713 entries, 1 to 146622
Data columns (total 81 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   VAERS_ID                    106713 non-null  int32  
 1   AGE_YRS                     106713 non-null  float64
 2   SEX                         106713 non-null  int32  
 3   VAX_MANU                    106713 non-null  int32  
 4   VAX_DOSE_SERIES             106713 non-null  int32  
 5   Headache                    106713 non-null  int64  
 6   Chills                      106713 non-null  int64  
 7   Pyrexia                     106713 non-null  int64  
 8   Fatigue                     106713 non-null  int64  
 9   Pain                        106713 non-null  int64  
 10  Nausea                      106713 non-null  int64  
 11  Dizziness                   106713 non-null  int64  
 12  Pain in extremity           106713 non-null  int64  
 13  Injection site

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import math 

df = shuffle(df_asn)

# print("70% of data set used:", math.floor(df.shape[0]*0.7))
log = LogisticRegression()
X = df[['AGE_YRS','SEX','VAX_MANU','VAX_DOSE_SERIES']]


Y_list = symptoms_to_use

# initialize list of models
model_list = list(map(lambda _ : LogisticRegression(), Y_list))

symptoms_dict ={}
for i, y in enumerate(Y_list):   
    Y= df[y]
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size =0.3)
    model_list[i].fit(x_train, y_train)
    symptoms_dict[y] = model_list[i].predict_proba(np.array([50,0,1, 1]).reshape(1, -1))[0][1]
    print("\nProbability of", y, "happening: ",model_list[i].predict_proba(np.array([50,0,1, 1]).reshape(1, -1))[0][1])
    print("Accuracy:", model_list[i].score(x_test, y_test))

# correct predictions / total number of data points
# print("Accuracy of this prediction: ",score/num)


Probability of Headache happening:  0.1688124911600698
Accuracy: 0.8310739051664897

Probability of Chills happening:  0.1365799135424204
Accuracy: 0.8646842006622103

Probability of Pyrexia happening:  0.13357253608631156
Accuracy: 0.8620291122633847

Probability of Fatigue happening:  0.1289621243371458
Accuracy: 0.8733679015430749

Probability of Pain happening:  0.12489903700926859
Accuracy: 0.8744611732367089

Probability of Nausea happening:  0.09375026074299736
Accuracy: 0.9063847066908227

Probability of Dizziness happening:  0.07926549305987882
Accuracy: 0.9209408383832074

Probability of Pain in extremity happening:  0.07246068151037767
Accuracy: 0.9225651277566065

Probability of Injection site pain happening:  0.0717634860134823
Accuracy: 0.9263759605172737

Probability of Injection site erythema happening:  0.06779443335658338
Accuracy: 0.9319672643218592

Probability of Myalgia happening:  0.05706534983222656
Accuracy: 0.9418379458986693

Probability of Arthralgia happen

In [8]:
# Top 10 most likely symptoms
sorted_dict = {}
sorted_keys = sorted(symptoms_dict, key=symptoms_dict.get, reverse = True)

# Get symptoms with occurences over 1000
for x in sorted_keys:
    sorted_dict[x] = symptoms_dict[x]

num = 0
print("These are the 10 mostly likely syptoms you would get")
for x in sorted_dict:
    if num == 10:
        break
    print("Probability of",x , "happening: ", sorted_dict[x])
    num += 1

These are the 10 mostly likely syptoms you would get
Probability of Headache happening:  0.1688124911600698
Probability of Chills happening:  0.1365799135424204
Probability of Pyrexia happening:  0.13357253608631156
Probability of Fatigue happening:  0.1289621243371458
Probability of Pain happening:  0.12489903700926859
Probability of Nausea happening:  0.09375026074299736
Probability of Dizziness happening:  0.07926549305987882
Probability of Pain in extremity happening:  0.07246068151037767
Probability of Injection site pain happening:  0.0717634860134823
Probability of Injection site erythema happening:  0.06779443335658338


In [9]:
import joblib
import os

BASE_DIR = "../../src/models/symptoms-sklearn/"

# clear dir
for file in os.listdir(BASE_DIR):
    os.remove(os.path.join(BASE_DIR, file))

# save models
for i, model in enumerate(model_list):
    model_name_raw = Y_list[i]
    model_name = model_name_raw.replace(" ", "_").upper()
    model_path = f"{BASE_DIR}model_{model_name}.pkl"
    joblib.dump(model, model_path)

joblib.dump(encoder_SEX, os.path.join(BASE_DIR, "encoder_SEX.pkl"))
joblib.dump(encoder_VAX_MANU, os.path.join(BASE_DIR, "encoder_VAX_MANU.pkl"))

['../../src/models/symptoms-sklearn/encoder_VAX_MANU.pkl']

In [10]:
# load model from filesystem
model_ = joblib.load(os.path.join(BASE_DIR, "model_HEADACHE.pkl"))
model_

LogisticRegression()

In [11]:
sample_ = x_train.iloc[0]
print(sample_)
sample_array_ = np.array(sample_)
sample_array_

AGE_YRS            77.0
SEX                 1.0
VAX_MANU            2.0
VAX_DOSE_SERIES     2.0
Name: 8589, dtype: float64


array([77.,  1.,  2.,  2.])

In [12]:
model_.predict_proba([sample_array_])

array([[0.82983818, 0.17016182]])